Introduction




This project is built around the idea of using large language models to assist in decision-making for fraud detection and refund  related cases. Having worked in Amazon  I have seen how much manual effort goes into analyzing a customers issue by frontline associates checking order details, product value, history of claims, and other documentation  before deciding whether to issue a refund or request supporting documents like a police or incident report.

The goal of this project was to fine-tune a pre-trained language model so that it can understand such case summaries and provide a risk-based recommendation (for example, “Low Risk,” “Medium Risk,” or “High Risk”). The model learns from real patterns of decision-making and applies those learned insights to new cases.

For this purpose, I used frameworks such as Unsloth, Hugging Face Transformers, Datasets, and TRL (Transformers Reinforcement Learning). The model was fine-tuned using LoRA optimization to make the process faster and more memory efficient. The dataset was prepared and processed using pandas and loaded into a Hugging Face-compatible format for training in Google Colab.

Through this project, my aim was to replicate how an experienced associate reasons through a refund decision  essentially creating a model that can act as an intelligent assistant, reducing human workload while improving consistency and speed in judgment.

In [ ]:
#importing necessary packages

!pip install -q pandas tqdm datasets

import pandas as pd, random, json

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df = pd.read_csv("Customer_DF (1).csv")




In [ ]:
df.is_fraud.value_counts()

In [ ]:
df.head()

In [ ]:
print(" Data loaded with shape:", df.shape)


I will convert the above table in a json format which is required for TRL library and hugging face dataset

Its kind of a prompt and response type as below

{
  "text_summary": "Customer from CA placed 2 orders, made 1 payment, had 2 transactions. Device: yyeiaxpltf82440jnb3v, IP: 8.129.104.40. Overall behavior appears legitimate.",

  
  "abuse": "Low risk. Approve refund normally."
}

In [ ]:
rename_map = {
    'No_Transactions': 'refund_count',
    'No_Orders': 'order_count',
    'No_Payments': 'payment_count',
    'Fraud': 'is_fraud'
}
df.rename(columns=rename_map, inplace=True)

In [ ]:
# row = df.iloc[0]

# row['customerDevice']

Feature Engineering for my  LLM fine-tuning to convert to json

In [ ]:
def make_summary(row):
    text = (
        f"Customer using device {row['customerDevice']} from IP {row['customerIPAddress']} "
        f"placed {row['order_count']} orders with {row['refund_count']} transactions "
        f"and made {row['payment_count']} payments. "
        f"Billing address: {row['customerBillingAddress']}. "
    )
    # adding signals
    if row['refund_count'] >= 3:
        text += "Frequent transaction activity detected. "
    if row['payment_count'] < row['order_count']:
        text += "Payment irregularities observed. "
    tone = "suspicious" if row['is_fraud'] else "legitimate"
    text += f"Overall behavior appears {tone}."
    return text


In [ ]:
def make_abuse(row):
    if row['is_fraud'] and row['refund_count'] >= 3:
        return "High risk refund abuse. Require Police Report."
    elif row['is_fraud']:
        return "Medium risk refund abuse. Require Incident Report."
    else:
        return "Low risk. Approve refund normally."

In [ ]:
from tqdm import tqdm # shows progress bar

tqdm.pandas()
df["text_summary"] = df.progress_apply(make_summary, axis=1)
df["abuse"] = df.progress_apply(make_abuse, axis=1)

with open("fraud_cases.json", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "text_summary": row["text_summary"],
            "abuse": row["abuse"]
        }, f)
        f.write("\n")



In [ ]:
!head -n 3 fraud_cases.json  #fine-tuning library datasets, trl in hugging face  needs a text file

Stage 2 Finetuning

In [ ]:

from unsloth import FastLanguageModel
import torch


In [ ]:
import math
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from unsloth import is_bfloat16_supported   #brain float

In [ ]:
# 2) Load dataset (JSON format)
dataset = load_dataset("json", data_files="fraud_cases.json", split="train")
print("Dataset example:", dataset[0])

In [ ]:
# Load tokenizer & base model
MODEL_NAME = "gpt2-medium"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token  #sequences = same length gpt 2 cannot handle automatically it was trained as a causal LM (predict next token),
# that is predict next token given all other token

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    load_in_8bit=True,  # efficient mixed-precision loading
    device_map="auto"
)

#gpt 2 medium 355 million parameters

In [ ]:
model = prepare_model_for_kbit_training(model)

print(" Model and tokenizer loaded in 8-bit mode successfully.")

In [ ]:
config=model.config
print(config)

In [ ]:
from peft import LoraConfig, get_peft_model


GPT 2 Medium has 24 transformer blocks nx layers , each with 16 self-attention heads. Each head operates over a 64-dimensional subspace, giving a hidden size of 1024. The total parameter count 345M  comes from attention weights, feed-forward projections, and embeddings


# I target the three core Linear layers:
#   c_attn → combined Q, K, V projection
#   c_fc   → MLP expansion
#   c_proj → output projection (both in attention & MLP)

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r = 8,#rank hyperparamter
    lora_alpha = 32, # a higher alpha value will assign more weight to the LoRA activations  alpha/rank which is delata  a scaling paramter i.e delta r
    target_modules = ["c_attn", "c_fc", "c_proj"],  # my traget modules in transformer architecture
    lora_dropout = 0.05, # dropout regulirization
    bias = "none",#I can also keep it all or lora_only
    task_type = "CAUSAL_LM"# GPt 2 type
)


In [ ]:
model = get_peft_model(model, lora_config)
print(" LoRA adapters attached successfully.")
model.print_trainable_parameters()

In [ ]:
def tokenize_function(examples):
    """Format each case & response pair and tokenize."""
    merged_texts = []
    for summary, abuse in zip(examples["text_summary"], examples["abuse"]):
        text = f"### Case:\n{summary}\n\n### Response:\n{abuse}"
        merged_texts.append(text)

    tokens = tokenizer(
        merged_texts,
        truncation=True,
        padding="max_length",
        max_length=1024,
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
print(" Tokenization complete.")
print("Sample tokens:\n", tokenizer.decode(tokenized_dataset[0]["input_ids"][:200]))# i am seeing for sanity first 200 tokens

In [ ]:
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_dataset,# my data set
    dataset_text_field="text_summary",
    max_seq_length=1024,
    packing=False,# It can make training 5x faster for short sequences. concatenate short sequences into single examplesand often improves throughput
    args=TrainingArguments(
        output_dir="./fraud_investigator_model",
        per_device_train_batch_size=2, # The batch size per GPU/TPU core
        gradient_accumulation_steps=4,# Number of steps to perform befor each gradient accumulation

        # So my effective batch size = 8 (per step)
        learning_rate=2e-4,# controlling gradient update step
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        max_steps=100, #Stopping training after these many steps.
        warmup_steps=5, #Number of steps during which LR linearly increases from 0  LR valuePrevents early overshooting.


        logging_steps=5,# Printting  metrics (loss, accuracy, etc.) every 5 steps.
        optim="adamw_8bit", # Optimizer adam
        weight_decay=0.01,# L2 regulirazrion discourages huge weights
        lr_scheduler_type="linear", #learning rate scheduler controls how the learning rate changes during training.

        #types like "constant",   "linear"  , polynomial
        # Linear is like  Starts high and then  linearly decreases to 0
        report_to="none",  # can be used  for obervability in tensorboard
        seed=1997, #for reproducibilty
        save_strategy="epoch",
        save_total_limit=2
    ),
)



In [ ]:
trainer_stats = trainer.train()
final_loss = trainer_stats.training_loss
perplexity = math.exp(final_loss)

print(f"Training complete — Final Loss: {final_loss:.4f}, Perplexity: {perplexity:.4f}")


In [ ]:
#My Training is  complete the Final Loss: 0.7582, Perplexity: 2.1344

#training_loss is the average negative log-likelihood per token across training steps.

#perplexity = math.exp(final_loss) gives intuition


# A perplexity of  around 2.1344  means that, on average, the model is choosing between fewer than 2 plausible tokens at each step → showing high confidence.

# Unlike regression tasks, the goal here isn’t driving loss toward zero, but ensuring a steady downward trend in loss and perplexity, paired with qualitative improvements in reasoning outputs.

In [ ]:
#saving model in my drive

from google.colab import drive
drive.mount('/content/drive')

# Copying  the  final fine-tuned model folder to my  Google Drive
!cp -r ./fraud_investigator_model /content/drive/MyDrive/
print(" Fine-tuned model successfully saved to Google Drive at:")
print("/content/drive/MyDrive/fraud_investigator_model/final_model")

In [ ]:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# later on if i plan to do inference
# Mounting my  Drive
# from google.colab import drive
# drive.mount('/content/drive')

# #Loading  model & tokenizer from Drive
# model_path = "/content/drive/MyDrive/fraud_investigator_model"

# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

# #  Creating  pipeline for inference
# fraud_detector = pipeline("text-generation", model=model, tokenizer=tokenizer)


In [ ]:

# doing a little inference to check it from my previous pipeline that was trained
from transformers import pipeline
pipe = pipeline(
    "text-generation",
    model="./fraud_investigator_model/final_model",
    tokenizer=tokenizer,
    device_map="auto"
)

prompt = """### Case:
Customer using device z8x31pq from IP 192.55.44.10 has placed 9 orders in the past 10 days.
Out of these, 1 were refunded citing “item not received”.
Multiple accounts share the same billing address: 44 Maple Street, San Diego, CA.
Payment method changed twice in one week.

### Response:"""

output = pipe(prompt, max_new_tokens=150, temperature=1.2)# i can twek temp to have more detailed output
print(" Model Prediction:\n", output[0]["generated_text"])

In [ ]:

 Model Prediction:
 ### Case:
Customer using device z8x31pq from IP 192.55.44.10 has placed 9 orders in the past 10 days.
Out of these, 1 were refunded citing “item not received”.
Multiple accounts share the same billing address: 44 Maple Street, San Diego, CA.
Payment method changed twice in one week.

### Response:
Low risk. Approve refund with issue noted.

My conclusion

This fine-tuning project demonstrates how a domain-specific language model can learn the reasoning patterns behind real-world decision-making processes. The model acts as a foundation for what could evolve into a more advanced, agentic system — one that not only interprets cases but also takes action based on learned logic.

I may give it access to tools and crm to update dashboard too

In the future, this concept can be extended by integrating the fine-tuned model with APIs or backend systems to create a human-in-the-loop agent. Such an agent could automatically read case descriptions, evaluate them, and perform tasks like issuing refunds, denying requests, or requesting documents, while still allowing a human reviewer to approve or override the decision.This make the concept Human in the loop

The broader vision is to move toward automation — where language models do not just generate responses but assist in real operational workflows with accountability, speed, and accuracy. This project serves as a first step toward that goal, blending human understanding with machine intelligence to make front facing processes smarter and more efficient.